In [7]:
import sys

sys.path.append("..")
from cge_modeling import Variable, Parameter, Equation, CGEModel
from cge_modeling.pytensorf.compile import (
    compile_cge_model_to_pytensor,
    euler_approximation_from_CGEModel,
    pytensor_objects_from_CGEModel,
)

In [2]:
var_list = [
    Variable(name="Y", description="Output", positive=True),
    Variable(name="C", description="Consumption", positive=True),
    Variable(name="INC", description="Household income", positive=True),
    Variable(name="L_d", description="Firm demand for labor", positive=True),
    Variable(name="K_d", description="Firm demand for capital", positive=True),
    Variable(name="r", description="Rental rate of capital", positive=True),
    Variable(name="w", description="Wage level", positive=True),
    Variable(name="P", description="Price level", positive=True),
    Variable(name="resid", latex_name="varepsilon", description="Walrasian residual"),
]

param_list = [
    Parameter(name="L_s", description="Household labor supply"),
    Parameter(name="K_s", description="Household capital supply"),
    Parameter(name="A", description="Total factor productivity"),
    Parameter(name="alpha", description="Share of capital in production"),
    Parameter(name="w_bar", latex_name=r"\bar{w}", description="Numeraire wage level"),
]

equation_list = [
    # Firm equations
    Equation(name="Production function", equation="Y - A * K_d ** alpha * L_d ** (1 - alpha)"),
    Equation(name="Capital demand", equation="K_d - alpha * Y * P / r"),
    Equation(name="Labor demand", equation="L_d - (1 - alpha) * Y * P / w"),
    # Household equatons
    Equation(name="Household income", equation="INC - w * L_s - r * K_s"),
    Equation(name="Household budget constraint", equation="P * C - INC"),
    # Equlibrium conditions
    Equation(name="Economic resource constraint", equation="Y - C"),
    Equation(name="Labor market clearing", equation="L_s - L_d + resid"),
    Equation(name="Capital market clearing", equation="K_s - K_d"),
    Equation(name="Numeraire wage", equation="w - w_bar"),
]

In [3]:
mod = CGEModel(
    variables=var_list,
    parameters=param_list,
    equations=equation_list,
    parse_equations_to_sympy=False,
)

In [4]:
mod.summary()

<IPython.core.display.Latex object>

In [5]:
mod_funcs = compile_cge_model_to_pytensor(mod)

In [10]:
pytensor_objects_from_CGEModel(mod)[1].type.shape

(9,)